In [133]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install tqdm
!pip install requests
!pip install arrow

In [134]:
import pandas as pd
import facebook
import requests
import arrow
from tqdm import tqdm

In [135]:
# Read env variables
ENV = pd.read_json('.env', orient='records').iloc[0]

In [136]:
graph = facebook.GraphAPI(access_token = ENV['ACCESS_TOKEN'], version = '2.7')

In [137]:
arw = arrow.utcnow()
current_month = arw.month

def extract_feed_data(feed, rows = []):
    should_finish = False

    for post in tqdm(feed['data'], 'Extracting current month data...'):
        post_month = arrow.get(post['created_time']).month
        if(post_month != current_month):
            should_finish = True
            break
        
        likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
        reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
        comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

        row_data = {
            'id': post['id'],
            'created_time': post['created_time'],
            'date': arrow.get(post['created_time']).format('DD-MM-YYYY'),
            'permalink_url': post['permalink_url'],
            'type': post['type'],
            'status_type': post['status_type'],
            'message': post['message'] if 'message' in post else '-',
            'total_likes': likes['summary']['total_count'],
            'total_reactions': reactions['summary']['total_count'],
            'total_comments': comments['summary']['total_count'],
            'total_shares': post['shares']['count'] if 'shares' in post else 0
        }
        rows.append(row_data)

    if(should_finish):
        print('Month data complete!')
        return rows
    else:
        next_feed = requests.get(feed['paging']['next']).json()
        return extract_feed_data(next_feed, rows)

In [138]:
def get_page_feed_data(page):
    # Extract feed
    page_data = pd.DataFrame(extract_feed_data(page['feed']))

    # Sort page_data by 'created_time'
    page_data.sort_values(['created_time'], ascending = True, inplace = True)
    page_data.index = range(1, len(page_data) + 1)

    # Add page name
    page_name = pd.Series(page['name'], index = page_data.index, name = 'page')
    page_data = page_data.join(page_name)
    
    return page_data

In [139]:
def get_page_object(page_id):
    return graph.get_object(
        id = page_id,
        fields='name, about, category, fan_count, feed.limit(100){created_time, permalink_url, type, status_type, message, shares}'
    )

In [140]:
# Get pages objects
page_one = get_page_object(ENV['PAGE_IDS'][0])
page_two = get_page_object(ENV['PAGE_IDS'][1])

# Get pages feed data
page_feed_data_one = get_page_feed_data(page_one)
page_feed_data_two = get_page_feed_data(page_two)

# Concat pages data
pages_feed_data = pd.concat([page_feed_data_one, page_feed_data_two], ignore_index = True)


Extracting current month data...:   0%|          | 0/100 [00:00<?, ?it/s]

Month data complete!



Extracting current month data...:  12%|█▏        | 12/100 [00:17<02:05,  1.42s/it]

Month data complete!


In [141]:
pages_feed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 12 columns):
created_time       270 non-null object
date               270 non-null object
id                 270 non-null object
message            270 non-null object
permalink_url      270 non-null object
status_type        270 non-null object
total_comments     270 non-null int64
total_likes        270 non-null int64
total_reactions    270 non-null int64
total_shares       270 non-null int64
type               270 non-null object
page               270 non-null object
dtypes: int64(4), object(8)
memory usage: 25.4+ KB


In [142]:
pages_feed_data.tail()

,created_time,date,id,message,permalink_url,status_type,total_comments,total_likes,total_reactions,total_shares,type,page
265,2017-04-22T12:00:00+0000,22-04-2017,173450952675280_1405660629454300,"Neste dia, no ano de 1500, a frota comandada p...",https://www.facebook.com/universidadepotiguar/...,added_photos,1,36,38,3,photo,UnP – Universidade Potiguar
266,2017-04-22T15:00:00+0000,22-04-2017,173450952675280_1405660799454283,Na próxima terça-feira (25/04) acontece o Work...,https://www.facebook.com/universidadepotiguar/...,added_photos,0,8,8,0,photo,UnP – Universidade Potiguar
267,2017-04-22T18:00:00+0000,22-04-2017,173450952675280_1405673686119661,Estaria no meio de muitos cálculos? Em um plan...,https://www.facebook.com/universidadepotiguar/...,added_photos,15,25,25,0,photo,UnP – Universidade Potiguar
268,2017-04-22T21:00:00+0000,22-04-2017,173450952675280_1405661482787548,"""Informática Cidadã"" é um projeto da UnP volta...",https://www.facebook.com/universidadepotiguar/...,shared_story,0,42,45,2,link,UnP – Universidade Potiguar
269,2017-04-23T00:00:00+0000,23-04-2017,173450952675280_1405661599454203,Se você deseja melhorar as suas habilidades de...,https://www.facebook.com/universidadepotiguar/...,added_photos,1,18,23,8,photo,UnP – Universidade Potiguar


In [143]:
from bokeh.charts import Scatter, Bar
from bokeh.io import output_notebook, show

In [144]:
p = Scatter(
        page_feed_data_one,
        x='date',
        y='total_reactions', 
        xlabel='Date',
        ylabel='Total Reactions', 
        color='type',
        title='Date vs Total Reactions - ' + page_one['name']
)
output_notebook()
show(p)

Loading BokehJS ...

In [146]:
p = Scatter(
        page_feed_data_two,
        x='date',
        y='total_reactions', 
        xlabel='Date',
        ylabel='Total Reactions', 
        color='type',
        title='Date vs Total Reactions - ' + page_two['name']
)
output_notebook()
show(p)

Loading BokehJS ...

In [147]:
p = Bar(
    pages_feed_data,
    label='date',
    values='total_reactions',
    agg='sum',
    stack='page',
    xlabel='Date',
    ylabel='Total Reactions',
    title="Sum Post Total Reactions by Date, stacked by Page",
    legend='top_right'
)
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
# References

# https://facebook-sdk.readthedocs.io/en/latest/api.html
# https://developers.facebook.com/docs/graph-api/reference/v2.7